In [1]:
import numpy as np
import collections
import heapq


In [3]:
initial_layout = [['📙','📘','📗','📘','📙','📗','📙'],
                  ['📕','📗','📕','📘','📕','📘','📙'],
                  ['📙','📘','📗','📗','📙','📘','📗'],
                  ['📘','📗','📘','📘','📙','📕','📕'],
                  ['📕','📗','📘','📙','📙','📘','📘'],
                  ['📙','📘','📗','📕','📙','📘','📙'],
                  ['📕','📕','📙','📕','📘','📙','📗'],
                  ['📗','📗','📕','📗','📗','📙','📗'],
                  ['📗','📕','📘','📘','📘','📕','📘']]
layout_dict = {'📙':1,
               '📘':2,
               '📗':3,
               '📕':4,}
print_dict = {1:'📙',
               2:'📘',
               3:'📗',
               4:'📕',
               0:'  '}

In [4]:
game_dict = {}
board_size = np.array(initial_layout).shape
for i in range(board_size[0]):
    for j in range(board_size[1]):
        number = layout_dict[initial_layout[i][j]]
        game_dict[i,j] = number

In [ ]:
class GraphEnvironment():

    def __init__(self,initial_layout):
        super().__init__()
        layout_dict = {'📙':1,
                       '📘':2,
                       '📗':3,
                       '📕':4,}
        
        board_size = np.array(initial_layout).shape

        action_map = {}
        game_array = np.zeros(board_size)
        action_number = 0
        for i in range(board_size[0]):
            for j in range(board_size[1]):
                number = layout_dict[initial_layout[i][j]]
                game_array[i,j] = number
                action_map[action_number] = [i,j]
                action_number += 1

        self.action_space = np.zeros(shape=(len(action_map)))
        self.obaservation_space = np.zeros(shape = (board_size[0]*board_size[1]))
        self.episode_ended = False
        
        self.initial_game_array = game_array.astype(int).copy()
        self.game_array = game_array.astype(int).copy()
        self.board_size = board_size
        self.action_map = action_map
        self.n_actions = 0
        self.n_remaining = np.sum(self.game_array!=0)    



    def step(self, game_array_code, action):
        
        self.game_array = np.array(list(int(c) for c in game_array_code)).reshape(board_size[0],board_size[1])

        def clear_tiles(self,i,j,clear_tile):
            #print(i,j)
            if i<0 or i>=board_size[0] or j < 0 or j>=board_size[1]:
                return
            tile = self.game_array[i,j]
            if tile != clear_tile or clear_tile==0:
                return
            #print('cleared')
            self.game_array[i,j] = 0
            clear_tiles(self,i-1,j,clear_tile)
            clear_tiles(self,i+1,j,clear_tile)
            clear_tiles(self,i,j-1,clear_tile)
            clear_tiles(self,i,j+1,clear_tile)

        def update_board(self):
            for j in range(board_size[1]):
                empty_spaces = 0
                for i in range(board_size[0]-1,-1,-1):
                    if self.game_array[i,j] == 0:
                        empty_spaces +=1
                    elif empty_spaces>0:
                        self.game_array[i+empty_spaces,j] = self.game_array[i,j]
                        self.game_array[i,j] = 0

        i = self.action_map[int(action)][0]
        j = self.action_map[int(action)][1]
        clear_tile = int(self.game_array[i,j])
        
        if clear_tile!=0:
            clear_tiles(self,i,j,clear_tile)
            update_board(self)

        next_state = self.game_array.reshape(-1,1).astype(int)[:,0]
        
        code = ''
        for c in next_state:
            code += str(c)
        return code
    
    def calc_heuristic(self,game_array_code):
        self.game_array = np.array(list(int(c) for c in game_array_code)).reshape(board_size[0],board_size[1])
        n_tiles = np.sum(self.game_array!=0)
        if True:
            groups_dict = collections.defaultdict(int)
            
            n_moves = collections.defaultdict(int)
            self.game_array = np.array(list(int(c) for c in game_array_code)).reshape(board_size[0],board_size[1])
            unique_tiles = set(game_array_code) - set('0')
            print(unique_tiles)
            for i in unique_tiles:
                column_list = np.any(self.game_array==int(i),axis=0)
                value = column_list[0]
                if value:
                    moves = 1
                    on_tile = True
                else:
                    moves = 0
                    on_tile = False
                for c in column_list[1:]:
                    if c != value:
                        on_tile = False
                    
                    if on_tile is False and c is True:
                        moves += 1
                n_moves[i] = moves
            print(n_moves)
        def clear_tiles(self,i,j,clear_tile):
            #print(i,j)
            if i<0 or i>=board_size[0] or j < 0 or j>=board_size[1]:
                return
            tile = self.game_array[i,j]
            if tile != clear_tile or clear_tile==0:
                return
            #print('cleared')
            self.game_array[i,j] = 0
            clear_tiles(self,i-1,j,clear_tile)
            clear_tiles(self,i+1,j,clear_tile)
            clear_tiles(self,i,j-1,clear_tile)
            clear_tiles(self,i,j+1,clear_tile)
        if False:
            current_tiles = n_tiles.copy()
            group_size = []
            for action in self.action_map.keys():
            
                i = self.action_map[int(action)][0]
                j = self.action_map[int(action)][1]
                clear_tile = int(self.game_array[i,j])
                
                if clear_tile!=0:
                    clear_tiles(self,i,j,clear_tile)
                    tile_count = np.sum(self.game_array!=0)

                    group_size.append(current_tiles - tile_count)
                    current_tiles = tile_count

            #print(group_size)
            average_group_size = np.mean(np.array(group_size))+5
        #print(average_group_size)
        if False:
            min_tile = min(groups_dict,key=groups_dict.get)
            #print(min_tile)
            unique_tiles = unique_tiles - set(str(min_tile))
            #print(groups_dict)
            #print(n_moves)
            for i in unique_tiles:
                groups_dict[i] = max(min(groups_dict[i],n_moves[i]),1)

            #print(groups_dict)

        return sum(n_moves.values())

In [95]:
initial_layout

[['📙', '📘', '📗', '📘', '📙', '📗', '📙'],
 ['📕', '📗', '📕', '📘', '📕', '📘', '📙'],
 ['📙', '📘', '📗', '📗', '📙', '📘', '📗'],
 ['📘', '📗', '📘', '📘', '📙', '📕', '📕'],
 ['📕', '📗', '📘', '📙', '📙', '📘', '📘'],
 ['📙', '📘', '📗', '📕', '📙', '📘', '📙'],
 ['📕', '📕', '📙', '📕', '📘', '📙', '📗'],
 ['📗', '📗', '📕', '📗', '📗', '📙', '📗'],
 ['📗', '📕', '📘', '📘', '📘', '📕', '📘']]

In [96]:
np.array(initial_layout)[7:,:]

array([['📗', '📗', '📕', '📗', '📗', '📙', '📗'],
       ['📗', '📕', '📘', '📘', '📘', '📕', '📘']], dtype='<U1')

In [ ]:
g_env = GraphEnvironment(np.array(initial_layout)[5:,:])
action_list = g_env.action_map
board_size = g_env.board_size
code = ''
for c in g_env.game_array.reshape(-1,1).astype(int)[:,0]:
    code += str(c)

initial_code = code.copy()
goal_code = '0'*len(code)
checked_nodes = set()
unchecked_nodes = set([code])
i = 0
len_unchecked = len(unchecked_nodes)

# A star algorithm: f = g+h
g_scores = collections.defaultdict(lambda: int(100))
f_scores = collections.defaultdict(lambda: int(100))

g_scores[code] = 0
f_scores[code] = 0

priority_list = []
heapq.heappush(priority_list,(f_scores[code],code))
best_g = len(code)

cameFrom = dict()
while len(priority_list)>0:
    current = heapq.heappop(priority_list)
    code = current[1]
    f = current[0]
    #print(f)
    code_set = set([code])
    if code == goal_code:
        print('Made it!')
        break
    
    n_tiles = sum(c!='0' for c in code)
    i+=1
    if n_tiles<best_g:
        print('NEW BEST',n_tiles,g_scores[code],f_scores[code])
        print(np.array(list(int(c) for c in code)).reshape(board_size[0],board_size[1]))
        best_g = n_tiles

    for action in range(len(action_list)):
        if code[action] != '0':
            new_code = g_env.step(code,action)

            tentative_g = g_scores[code] + 1
            if tentative_g < g_scores[new_code]:
                #h1 = sum(c!='0' for c in new_code)
                #h2 = len(set(new_code)-set('0'))
                #h = g_env.calc_heuristic(new_code)
                h = 0
                
                cameFrom[new_code] = code
                g_scores[new_code] = tentative_g
                f_scores[new_code] = tentative_g + h

                heapq.heappush(priority_list,(f_scores[new_code],new_code))

    
    

NEW BEST 25 1 1
[[0 0 3 4 1 2 1]
 [0 2 1 4 2 1 3]
 [1 4 4 3 3 1 3]
 [4 4 2 2 2 4 2]]
NEW BEST 22 2 2
[[0 0 0 0 0 2 1]
 [0 2 3 4 1 1 3]
 [1 4 1 4 2 1 3]
 [4 4 4 3 3 4 2]]
NEW BEST 21 2 2
[[0 0 0 4 1 2 1]
 [0 0 3 4 2 1 3]
 [0 0 1 3 3 1 3]
 [1 2 2 2 2 4 2]]
NEW BEST 19 3 3
[[0 0 0 0 0 0 1]
 [0 2 3 4 0 0 3]
 [1 4 1 4 2 2 3]
 [4 4 4 3 3 4 2]]
NEW BEST 17 3 3
[[0 0 0 0 0 2 1]
 [0 0 0 4 1 1 3]
 [0 0 3 4 2 1 3]
 [1 0 1 3 3 4 2]]
NEW BEST 16 4 4
[[0 0 0 0 0 0 0]
 [0 2 3 4 0 0 0]
 [1 4 1 4 2 2 0]
 [4 4 4 3 3 4 2]]
NEW BEST 14 4 4
[[0 0 0 0 0 0 1]
 [0 0 0 4 0 0 3]
 [0 0 3 4 2 2 3]
 [1 0 1 3 3 4 2]]
NEW BEST 12 4 4
[[0 0 0 0 0 2 1]
 [0 0 0 0 0 1 3]
 [0 0 3 0 0 1 3]
 [1 0 1 0 1 4 2]]
NEW BEST 11 5 5
[[0 0 0 0 0 0 0]
 [0 0 0 4 0 0 0]
 [0 0 3 4 2 2 0]
 [1 0 1 3 3 4 2]]
NEW BEST 8 5 5
[[0 0 0 0 0 0 1]
 [0 0 0 0 0 0 3]
 [0 0 3 0 0 0 3]
 [1 0 1 0 0 4 2]]
NEW BEST 5 6 6
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 3 0 0 0 0]
 [1 0 1 0 0 4 2]]
NEW BEST 4 7 7
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 

In [106]:
g_env = GraphEnvironment(np.array(initial_layout)[5:,:])
action_list = g_env.action_map
board_size = g_env.board_size
code = ''
for c in g_env.game_array.reshape(-1,1).astype(int)[:,0]:
    code += str(c)

initial_code = code

In [89]:
np.array(initial_layout)[8:,:]

array([['📗', '📕', '📘', '📘', '📘', '📕', '📘']], dtype='<U1')

In [118]:
last_code = goal_code
reconstructed_path = []
#reconstructed_path.append(goal_code)
while last_code!= initial_code:
    last_code = cameFrom[last_code]
    reconstructed_path.append(np.array(list(int(c) for c in last_code)).reshape(g_env.board_size[0],g_env.board_size[1]))

In [119]:
reconstructed_path[-1:0:-1]

[array([[1, 2, 3, 4, 1, 2, 1],
        [4, 4, 1, 4, 2, 1, 3],
        [3, 3, 4, 3, 3, 1, 3],
        [3, 4, 2, 2, 2, 4, 2]]),
 array([[0, 0, 3, 4, 1, 2, 1],
        [0, 2, 1, 4, 2, 1, 3],
        [1, 4, 4, 3, 3, 1, 3],
        [4, 4, 2, 2, 2, 4, 2]]),
 array([[0, 0, 3, 0, 0, 2, 1],
        [0, 2, 1, 4, 1, 1, 3],
        [1, 4, 4, 4, 2, 1, 3],
        [4, 4, 2, 2, 2, 4, 2]]),
 array([[0, 0, 0, 0, 0, 2, 1],
        [0, 0, 3, 0, 1, 1, 3],
        [0, 0, 1, 0, 2, 1, 3],
        [1, 2, 2, 2, 2, 4, 2]]),
 array([[0, 0, 0, 0, 0, 2, 0],
        [0, 0, 3, 0, 1, 1, 0],
        [0, 0, 1, 0, 2, 1, 1],
        [1, 2, 2, 2, 2, 4, 2]]),
 array([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 3, 0, 0, 0, 0],
        [0, 0, 1, 0, 2, 2, 0],
        [1, 2, 2, 2, 2, 4, 2]]),
 array([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 2, 2, 0],
        [1, 2, 2, 2, 2, 4, 2]]),
 array([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 2, 0, 0],
        [1, 2, 2, 2, 2, 2

In [120]:
len(reconstructed_path)

10

In [102]:
g_scores[goal_code]

10

In [48]:
g_env = GraphEnvironment(np.array(initial_layout)[:,0:3])
action_list = g_env.action_map
board_size = g_env.board_size

code = ''
for c in g_env.game_array.reshape(-1,1).astype(int)[:,0]:
    code += str(c)

checked_nodes = set()
unchecked_nodes = set([code])
i = 0
len_unchecked = len(unchecked_nodes)
# Breadth first search
running_checked = []
while len(unchecked_nodes)>0:
    code = list(unchecked_nodes)[0]
    checked_nodes = checked_nodes.union(set([code]))
    running_checked.append(code)
    code_set = set([code])
    
    for action in range(len(action_list)):
        if code[action] != '0':
            new_code = g_env.step(code,action)
           
            if not new_code in checked_nodes:
                unchecked_nodes = unchecked_nodes.union(set([new_code]))

    unchecked_nodes = unchecked_nodes - set([code])
    i+=1
    if i %100 ==0:
        new_len = len(unchecked_nodes)
        print(i,len_unchecked,new_len-len_unchecked)
        len_unchecked = new_len



100 1 1271
200 1272 1073
300 2345 983
400 3328 896
500 4224 931
600 5155 960
700 6115 939
800 7054 811
900 7865 864
1000 8729 918
1100 9647 853
1200 10500 835
1300 11335 892
1400 12227 812
1500 13039 844
1600 13883 771
1700 14654 770
1800 15424 730
1900 16154 852
2000 17006 789
2100 17795 779
2200 18574 789
2300 19363 692
2400 20055 774
2500 20829 768
2600 21597 741
2700 22338 776
2800 23114 762
2900 23876 718
3000 24594 695


KeyboardInterrupt: 

In [33]:
len(checked_nodes)

11328

In [36]:
len(checked_nodes)

28513

In [38]:
len(checked_nodes)

11268

In [40]:
len(checked_nodes)

1364

In [42]:
len(checked_nodes)

2840

In [44]:
len(checked_nodes)

5816

In [45]:
11328+28513+11268+1364+2840+5816

61129